# Improving Test Writing, Reading, and Troubleshooting

> If you can’t make something self-evident, you at least need to make it self-explanatory.  
> ― Steve Krug, Don't Make Me Think

In [1]:
#r "nuget:FluentAssertions,5.10.3"
#r "nuget:NUnit,3.12.0"

int returnedValue = 5;

string helloWorld = "Hello, World!";
System.Collections.Generic.List<string> colors = new() { "red", "blue", "green" };

# Closely Matching Your Mental Model
What's the problem with `Assert.AreEqual(returnedValue, 4)`?

1. Doesn't match the usual mental developer model of `value == 4`. If anything, it's more like Reverse Polish Notation: `== value 4`.
2. Without tooltips, can you tell which one is is the `actual` parameters and which one is the `expected` parameter? It does influence the resulting message.  
  `expected` is actually the first parameter, so the *semantically* correct way to write that assert is really `Assert.AreEqual(4, returnedValue)`.

How would this look like in FluentAssertions: `returnedValue.Should().Be(4)`.

To be fair, `Assert.AreEqual` is NUnit 2 (called *Classic Model*). NUnit 3 assertions, called *Constraint Model* read a bit better: `Assert.That(returnedValue. Is.EqualTo(4))`.

# Reasons
Sometimes it may be entirely obvious why `returnedValue` should be precisely `4`, but it does reek of [Magic Number Anti-pattern](https://www.pluralsight.com/tech-blog/avoiding-magic-numbers/).

You can kind of do it in NUnit, and while familiar is still a bit clumsy. Compare:

In [1]:
using NUnit.Framework;
Assert.AreEqual(4, returnedValue, "We seeded 4 values");

Error: NUnit.Framework.AssertionException:   We seeded 4 values
  Expected: 4
  But was:  5

   at NUnit.Framework.Assert.ReportFailure(String message) in D:\a\1\s\src\NUnitFramework\framework\Assert.cs:line 394
   at NUnit.Framework.Assert.ReportFailure(ConstraintResult result, String message, Object[] args) in D:\a\1\s\src\NUnitFramework\framework\Assert.cs:line 382
   at NUnit.Framework.Assert.That[TActual](TActual actual, IResolveConstraint expression, String message, Object[] args) in D:\a\1\s\src\NUnitFramework\framework\Assert.That.cs:line 240
   at NUnit.Framework.Assert.AreEqual(Object expected, Object actual, String message, Object[] args) in D:\a\1\s\src\NUnitFramework\framework\Assert.Equality.cs:line 115
   at Submission#33.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 e

In [1]:
using FluentAssertions;
returnedValue.Should().Be(4, because: "that's how many values we seeded");

Error: NUnit.Framework.AssertionException: Expected value to be 4 because that's how many values we seeded, but found 5.
   at FluentAssertions.Execution.LateBoundTestFramework.Throw(String message) in C:\projects\fluentassertions-vf06b\Src\FluentAssertions\Execution\LateBoundTestFramework.cs:line 16
   at FluentAssertions.Execution.TestFrameworkProvider.Throw(String message) in C:\projects\fluentassertions-vf06b\Src\FluentAssertions\Execution\TestFrameworkProvider.cs:line 40
   at FluentAssertions.Execution.DefaultAssertionStrategy.HandleFailure(String message) in C:\projects\fluentassertions-vf06b\Src\FluentAssertions\Execution\DefaultAssertionStrategy.cs:line 29
   at FluentAssertions.Execution.AssertionScope.FailWith(Func`1 failReasonFunc) in C:\projects\fluentassertions-vf06b\Src\FluentAssertions\Execution\AssertionScope.cs:line 223
   at FluentAssertions.Execution.AssertionScope.FailWith(Func`1 failReasonFunc) in C:\projects\fluentassertions-vf06b\Src\FluentAssertions\Execut

# Specific Assertions
NUnit 2 assertions, which we use, have extremely basic functionality that we bend manually into accomplishing what we need, increasing mental cost.  
NUnit 3 enriches this domain, but still places a larger burned on the developer.

FluentAssertions in contrast, comes with a rich library of assertions. IMO, they're all fairly self-explanatory even to the casual reader.


## Strings Are Life
A good assert framework should have support for easy to read assertions on strings.

At the very basic equality, the two match. Beyond that, NUnit assertions fall quickly behind.

In [1]:
using NUnit.Framework;
Assert.AreEqual("Hello World", helloWorld);

Error: NUnit.Framework.AssertionException:   Expected string length 11 but was 13. Strings differ at index 5.
  Expected: "Hello World"
  But was:  "Hello, World!"
  ----------------^

   at NUnit.Framework.Assert.ReportFailure(String message) in D:\a\1\s\src\NUnitFramework\framework\Assert.cs:line 394
   at NUnit.Framework.Assert.ReportFailure(ConstraintResult result, String message, Object[] args) in D:\a\1\s\src\NUnitFramework\framework\Assert.cs:line 382
   at NUnit.Framework.Assert.That[TActual](TActual actual, IResolveConstraint expression, String message, Object[] args) in D:\a\1\s\src\NUnitFramework\framework\Assert.That.cs:line 240
   at NUnit.Framework.Assert.AreEqual(Object expected, Object actual) in D:\a\1\s\src\NUnitFramework\framework\Assert.Equality.cs:line 128
   at Submission#35.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArr

In [1]:
using FluentAssertions;
helloWorld.Should().Be("Hello World");

Error: NUnit.Framework.AssertionException: Expected string to be 
"Hello World" with a length of 11, but 
"Hello, World!" has a length of 13, differs near ", W" (index 5).
   at FluentAssertions.Execution.LateBoundTestFramework.Throw(String message) in C:\projects\fluentassertions-vf06b\Src\FluentAssertions\Execution\LateBoundTestFramework.cs:line 16
   at FluentAssertions.Execution.TestFrameworkProvider.Throw(String message) in C:\projects\fluentassertions-vf06b\Src\FluentAssertions\Execution\TestFrameworkProvider.cs:line 40
   at FluentAssertions.Execution.DefaultAssertionStrategy.HandleFailure(String message) in C:\projects\fluentassertions-vf06b\Src\FluentAssertions\Execution\DefaultAssertionStrategy.cs:line 29
   at FluentAssertions.Execution.AssertionScope.FailWith(Func`1 failReasonFunc) in C:\projects\fluentassertions-vf06b\Src\FluentAssertions\Execution\AssertionScope.cs:line 223
   at FluentAssertions.Execution.AssertionScope.FailWith(Func`1 failReasonFunc) in C:\projec

What about complex assertions?

1. String is null or empty;
2. Compare two strings case insensitive;
3. String contains another string;
4. Compare string ranges (beginning/end);
5. String matches expression or RegEx

### String Is Null or Empty, Null or Whitespace

In [1]:
using NUnit.Framework;
Assert.IsTrue(string.IsNullOrEmpty(helloWorld)); // => Terrible message Expected: True but was: False

Error: NUnit.Framework.AssertionException:   Expected: True
  But was:  False

   at NUnit.Framework.Assert.ReportFailure(String message) in D:\a\1\s\src\NUnitFramework\framework\Assert.cs:line 394
   at NUnit.Framework.Assert.ReportFailure(ConstraintResult result, String message, Object[] args) in D:\a\1\s\src\NUnitFramework\framework\Assert.cs:line 382
   at NUnit.Framework.Assert.That[TActual](TActual actual, IResolveConstraint expression, String message, Object[] args) in D:\a\1\s\src\NUnitFramework\framework\Assert.That.cs:line 240
   at NUnit.Framework.Assert.IsTrue(Boolean condition) in D:\a\1\s\src\NUnitFramework\framework\Assert.Conditions.cs:line 119
   at Submission#37.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, CancellationTo

In [1]:
using FluentAssertions;
helloWorld.Should().BeNullOrEmpty();

Error: NUnit.Framework.AssertionException: Expected string to be <null> or empty, but found "Hello, World!".
   at FluentAssertions.Execution.LateBoundTestFramework.Throw(String message) in C:\projects\fluentassertions-vf06b\Src\FluentAssertions\Execution\LateBoundTestFramework.cs:line 16
   at FluentAssertions.Execution.TestFrameworkProvider.Throw(String message) in C:\projects\fluentassertions-vf06b\Src\FluentAssertions\Execution\TestFrameworkProvider.cs:line 40
   at FluentAssertions.Execution.DefaultAssertionStrategy.HandleFailure(String message) in C:\projects\fluentassertions-vf06b\Src\FluentAssertions\Execution\DefaultAssertionStrategy.cs:line 29
   at FluentAssertions.Execution.AssertionScope.FailWith(Func`1 failReasonFunc) in C:\projects\fluentassertions-vf06b\Src\FluentAssertions\Execution\AssertionScope.cs:line 223
   at FluentAssertions.Execution.AssertionScope.FailWith(Func`1 failReasonFunc) in C:\projects\fluentassertions-vf06b\Src\FluentAssertions\Execution\Assertio

### Compare two string case insensitive

In [1]:
using NUnit.Framework;
Assert.AreEqual("YELLO, WORLD!", helloWorld.ToUpperInvariant()); // ugly but functional

In [1]:
using FluentAssertions;
helloWorld.Should().BeEquivalentTo("yello, world!");

### String contains another string

In [1]:
using NUnit.Framework;
// Assert.Contains is for collections, not strings

In [1]:
using FluentAssertions;
helloWorld.Should().Contain("; ");

### Compare string ranges or parts

In [1]:
using NUnit.Framework;
Assert.IsTrue(helloWorld.StartsWith("H") && helloWorld.EndsWith("!")); // ugly AND useless message

In [1]:
using FluentAssertions;
helloWorld.Should().StartWith("H").And.EndWith("?");

Error: NUnit.Framework.AssertionException: Expected string "Hello, World!" to end with "?".
   at FluentAssertions.Execution.LateBoundTestFramework.Throw(String message) in C:\projects\fluentassertions-vf06b\Src\FluentAssertions\Execution\LateBoundTestFramework.cs:line 16
   at FluentAssertions.Execution.TestFrameworkProvider.Throw(String message) in C:\projects\fluentassertions-vf06b\Src\FluentAssertions\Execution\TestFrameworkProvider.cs:line 40
   at FluentAssertions.Execution.DefaultAssertionStrategy.HandleFailure(String message) in C:\projects\fluentassertions-vf06b\Src\FluentAssertions\Execution\DefaultAssertionStrategy.cs:line 29
   at FluentAssertions.Execution.AssertionScope.FailWith(Func`1 failReasonFunc) in C:\projects\fluentassertions-vf06b\Src\FluentAssertions\Execution\AssertionScope.cs:line 223
   at FluentAssertions.Execution.AssertionScope.FailWith(Func`1 failReasonFunc) in C:\projects\fluentassertions-vf06b\Src\FluentAssertions\Execution\AssertionScope.cs:line 19

### More complex string matching rules

In [1]:
using NUnit.Framework;
// some variant of IsTrue

In [1]:
using FluentAssertions;
helloWorld.Should().Match("Hello*World?*");

In [1]:
using FluentAssertions;
helloWorld.Should().MatchRegex(@"\w \w");

## Collections Are Also Life
Collection validation may require complex rules at time, but for 80% of the case the framework should provide sensible values.

Unfortunately NUnit falls terribly short in this chapter, putting the onus on the developer. Instead of easy to read and reason code, collection assertions become a game of mental compilation.

### Count of elements

In [1]:
using NUnit.Framework;
Assert.AreEqual(4, colors.Count()); // poor message if mixed with other numeric asserts

Error: NUnit.Framework.AssertionException:   Expected: 4
  But was:  3

   at NUnit.Framework.Assert.ReportFailure(String message) in D:\a\1\s\src\NUnitFramework\framework\Assert.cs:line 394
   at NUnit.Framework.Assert.ReportFailure(ConstraintResult result, String message, Object[] args) in D:\a\1\s\src\NUnitFramework\framework\Assert.cs:line 382
   at NUnit.Framework.Assert.That[TActual](TActual actual, IResolveConstraint expression, String message, Object[] args) in D:\a\1\s\src\NUnitFramework\framework\Assert.That.cs:line 240
   at NUnit.Framework.Assert.AreEqual(Object expected, Object actual) in D:\a\1\s\src\NUnitFramework\framework\Assert.Equality.cs:line 128
   at Submission#40.<<Initialize>>d__0.MoveNext()
--- End of stack trace from previous location ---
   at Microsoft.CodeAnalysis.Scripting.ScriptExecutionState.RunSubmissionsAsync[TResult](ImmutableArray`1 precedingExecutors, Func`2 currentExecutor, StrongBox`1 exceptionHolderOpt, Func`2 catchExceptionOpt, Cancella

In [1]:
using FluentAssertions;
colors.Should().HaveCount(4);

Error: NUnit.Framework.AssertionException: Expected collection to contain 4 item(s), but found 3.
   at FluentAssertions.Execution.LateBoundTestFramework.Throw(String message) in C:\projects\fluentassertions-vf06b\Src\FluentAssertions\Execution\LateBoundTestFramework.cs:line 16
   at FluentAssertions.Execution.TestFrameworkProvider.Throw(String message) in C:\projects\fluentassertions-vf06b\Src\FluentAssertions\Execution\TestFrameworkProvider.cs:line 40
   at FluentAssertions.Execution.DefaultAssertionStrategy.HandleFailure(String message) in C:\projects\fluentassertions-vf06b\Src\FluentAssertions\Execution\DefaultAssertionStrategy.cs:line 29
   at FluentAssertions.Execution.AssertionScope.FailWith(Func`1 failReasonFunc) in C:\projects\fluentassertions-vf06b\Src\FluentAssertions\Execution\AssertionScope.cs:line 223
   at FluentAssertions.Execution.AssertionScope.FailWith(Func`1 failReasonFunc) in C:\projects\fluentassertions-vf06b\Src\FluentAssertions\Execution\AssertionScope.cs:l

### Contains a certain element

In [1]:
using NUnit.Framework;
Assert.Contains("cyan", colors); // reads awkward

In [1]:
using FluentAssertions;
colors.Should().Contain("cyan");

Error: NUnit.Framework.AssertionException: Expected collection {"red", "blue", "green"} to contain "cyan".
   at FluentAssertions.Execution.LateBoundTestFramework.Throw(String message) in C:\projects\fluentassertions-vf06b\Src\FluentAssertions\Execution\LateBoundTestFramework.cs:line 16
   at FluentAssertions.Execution.TestFrameworkProvider.Throw(String message) in C:\projects\fluentassertions-vf06b\Src\FluentAssertions\Execution\TestFrameworkProvider.cs:line 40
   at FluentAssertions.Execution.DefaultAssertionStrategy.HandleFailure(String message) in C:\projects\fluentassertions-vf06b\Src\FluentAssertions\Execution\DefaultAssertionStrategy.cs:line 29
   at FluentAssertions.Execution.AssertionScope.FailWith(Func`1 failReasonFunc) in C:\projects\fluentassertions-vf06b\Src\FluentAssertions\Execution\AssertionScope.cs:line 223
   at FluentAssertions.Execution.AssertionScope.FailWith(Func`1 failReasonFunc) in C:\projects\fluentassertions-vf06b\Src\FluentAssertions\Execution\AssertionS

### Contains certain set

In [1]:
using NUnit.Framework;
// Assert.IsTrue(colors.Contains("blue") && colors.Contains("cyan")); // don't know which one failed
// alternative
Assert.Multiple(() => {
    Assert.Contains("blue", colors);
    Assert.Contains("cyan", colors);
});

In [1]:
using FluentAssertions;
colors.Should().Contain(new[] { "blue", "cyan" });

Error: NUnit.Framework.AssertionException: Expected collection {"red", "blue", "green"} to contain {"blue", "cyan"}, but could not find {"cyan"}.
   at FluentAssertions.Execution.LateBoundTestFramework.Throw(String message) in C:\projects\fluentassertions-vf06b\Src\FluentAssertions\Execution\LateBoundTestFramework.cs:line 16
   at FluentAssertions.Execution.TestFrameworkProvider.Throw(String message) in C:\projects\fluentassertions-vf06b\Src\FluentAssertions\Execution\TestFrameworkProvider.cs:line 40
   at FluentAssertions.Execution.DefaultAssertionStrategy.HandleFailure(String message) in C:\projects\fluentassertions-vf06b\Src\FluentAssertions\Execution\DefaultAssertionStrategy.cs:line 29
   at FluentAssertions.Execution.AssertionScope.FailWith(Func`1 failReasonFunc) in C:\projects\fluentassertions-vf06b\Src\FluentAssertions\Execution\AssertionScope.cs:line 223
   at FluentAssertions.Execution.AssertionScope.FailWith(Func`1 failReasonFunc) in C:\projects\fluentassertions-vf06b\Sr

### Contain a certain set in the same order
Nothing for NUnit, built-in for FA.

In [1]:
using FluentAssertions;
colors.Should().ContainInOrder(new[] { "blue", "red"});

### Contain only unique items
Nothing for NUnit, built-in for FA.

In [1]:
using FluentAssertions;
colors.Should().OnlyHaveUniqueItems();

### Validate that all elements respect a certain rule
Some variant on LINQ + `IsTrue` for NUnit. 

In [1]:
using FluentAssertions;
colors.Should().OnlyContain(x => x.Length > 3);

### Validate that elements are sorted

In [1]:
using FluentAssertions;
colors.Should().BeInAscendingOrder();

### Have elements sorted by a property

In [1]:
using FluentAssertions;
users.Should().BeInAscendingOrder(x => x.FirstName);

FluentAssertions supports many more [collection asserts](https://fluentassertions.com/collections/) that NUnit simply doesn't support:
* comparison of boxed objects and object graph comparison
* precedence / succession of elements
* intersection of collections
* matching of strings for string collections

## Dictionaries

FA contains expected tests for presence/absence of keys and values: `ContainsKey/ContainsKeys`, `ContainsValue/Values`, counts -- `HaveCount`, and all the other collection asserts when treating the dictionary as an `IEnumerable<KeyValuePair<TK, TV>>`

## Date and Times
FA has [convenience methods](https://fluentassertions.com/datetimespans/) that increase readability. Remember the awkward `expected`/`actual` order in NUnit? No need for guessing in FA:

```csharp
laborDay.Should().BeOnOrAfter(1.September(2021))
  .And.NotBeAfter(8.September(2021));

serverDate.Should().BeIn(DateTimeKind.Utc);

xmasDayOff.Should().BeOneOf(24.December(2021), 26.December(2021));

updatedOn.Should().BeWithin(2.Seconds()).After(executionStart);
endProcessTime.Should().BeCloseTo(DateTime.Now, TimeSpan.FromSeconds(1));
```

## Exceptions
NUnit supports basic checks for assertions and for the most part they are sufficient:  
`Assert.Throw<ArgumentNullException>(() => MethodThatThrows(param1, param2))`.

But what if we want or need to perform further asserts on the exception being thrown? FA to the rescue, although with a bit more verbose syntax:

```csharp
Action constructor = () => new ConstructorThatThrows(param1: "one", param2: null);
constructor.Should().Throw<ArgumentNullException>()
  .And.ParamName.Should.Be("param2");
```

Any other properties can be asserted on -- useful when exception discrimination is not class based:

```csharp
dbMethod.Should().Throw<SqlException>()
  .And.Number.Should().Be(2627, because: "we expected a primary key violation");
```

Messages can be tested as well and it supports the same string wildcards that string assertions do:

```csharp
httpCall.Should().Throw<HttpException>()
  .WithMessage("*BAD REQUEST*");
```

## Object Graph Tests
NUnit has some support for object testing, but it's so poorly documented that it's mostly a trial and error and then abandon it to write your own code.

FA supports structural equality of two objects with `Should().BeEquivalentTo()`. It's recursive by default up to 10 levels, but it can be disabled with `options.ExcludingNestedObjects()`. 

There is further tuning of whether something should be compared by value (using `Object.Equals` overrides, default) or by members; auto-conversion of types by matching complex object paths, member and subsets of members.

NUnit has nothing as sophisticated.

# The Rest

FA has support for a wide range of features that NUnit does not. In no particular order:


* More sophisticated numeric asserts: `BeInRange()`, `BeApproximately()`, `BeOneOf()`;
* XML asserts
* Event monitoring
* Execution time monitoring: `someAsyncWork.Should().CompleteWithin(100.Milliseconds())`;
* [Type, method, and property assertions](https://fluentassertions.com/typesandmethods/): `method.Should().BeVirtual()`, `typeof(SomeClass).Should().BeDecoratedWith<ProtoContract>()`, including some wide-matching class enforcement ability:
```csharp
typeof(MyController).Methods()
  .ThatReturn<ActionResult>()
  .ThatAreDecoratedWith<HttpPostAttribute>()
  .Should()
  .BeDecoratedWith<ValidateAntiForgeryTokenAttribute>(
    "because all Actions with HttpPost require ValidateAntiForgeryToken");
```
* Assembly reference checking;
* Assertion scopes so you can batch multiple assertions and get one exception with all the failures;

> Keep it simple, so you'll keep doing it.  
> ― Steve Krug, Don't Make Me Think